In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
words =open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars= sorted(list(set(''.join(words))))
stoi= {s:i+1 for i,s in enumerate(chars)}
stoi['.']= 0
itos = {i:s for s,i in stoi.items()}

In [5]:
block_size= 3
X,Y = [],[]
for w in words[:5]:
    print(w)
    context = [0]* block_size
    for ch in w + '.':
        ix= stoi [ch]
        X.append(context)
        Y.append(ix)
        print((''.join(itos[i] for i in context), '----->', itos[ix]))
        context = context[1:] + [ix]
X= torch.tensor(X)
Y= torch.tensor(Y)

emma
('...', '----->', 'e')
('..e', '----->', 'm')
('.em', '----->', 'm')
('emm', '----->', 'a')
('mma', '----->', '.')
olivia
('...', '----->', 'o')
('..o', '----->', 'l')
('.ol', '----->', 'i')
('oli', '----->', 'v')
('liv', '----->', 'i')
('ivi', '----->', 'a')
('via', '----->', '.')
ava
('...', '----->', 'a')
('..a', '----->', 'v')
('.av', '----->', 'a')
('ava', '----->', '.')
isabella
('...', '----->', 'i')
('..i', '----->', 's')
('.is', '----->', 'a')
('isa', '----->', 'b')
('sab', '----->', 'e')
('abe', '----->', 'l')
('bel', '----->', 'l')
('ell', '----->', 'a')
('lla', '----->', '.')
sophia
('...', '----->', 's')
('..s', '----->', 'o')
('.so', '----->', 'p')
('sop', '----->', 'h')
('oph', '----->', 'i')
('phi', '----->', 'a')
('hia', '----->', '.')


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C= torch.randn(27,2)

In [9]:
C[5]

tensor([0.3765, 0.1553])

In [12]:
C[torch.tensor([5,6,7])]

tensor([[ 0.3765,  0.1553],
        [-0.5639, -1.9406],
        [ 0.4813, -1.3383]])

In [18]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [22]:
torch.cat(torch.unbind(emb, 1),1).shape #less efficeint

torch.Size([32, 6])

In [24]:
emb.view(32,6)== torch.cat(torch.unbind(emb, 1),1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [26]:
W1=torch.randn (6,100)
B1=torch.randn(100)

In [29]:
h = torch.tanh(emb.view(-1,6) @ W1 + B1)

torch.Size([32, 100])

In [31]:
W2= torch.randn(100,27)
B2= torch.randn(27)
logits = h @ W2 + B2

In [ ]:
l